In [12]:
import heapq
import math
import time
import sys
import random


# Graph definition: Each node points to its neighbors with corresponding travel costs
campus_map = {
    "Admission Office": {"Hostel Office": 2, "Library": 4},
    "Hostel Office": {"Admission Office": 2, "Hostel Visit": 2, "Canteen": 6, "Library": 4},
    "Hostel Visit": {"Hostel Office": 2, "Canteen": 6, "Exit": 4},
    "Canteen": {"Hostel Visit": 6, "Hostel Office": 6, "Library": 7, "Dep't Visit": 2, "Exit": 8},
    "Dep't Visit": {"Canteen": 2, "Library": 3, "Exit": 5},
    "Library": {"Admission Office": 4, "Canteen": 7, "Dep't Visit": 3, "Hostel Office": 4},
    "Exit": {"Dep't Visit": 5, "Canteen": 8, "Hostel Visit": 4}
}

# Coordinates for Euclidean heuristic: used to calculate distances between locations
node_coordinates = {
    "Admission Office": (0, 7),
    "Hostel Office": (2, 8),
    "Hostel Visit": (9, 9),
    "Canteen": (7, 5),
    "Dep't Visit": (6, 0),
    "Library": (1, 3),
    "Exit": (12, 8)
}

# Function to calculate Euclidean distance between two nodes
def euclidean_heuristic(node, target):
    x1, y1 = node_coordinates[node]
    x2, y2 = node_coordinates[target]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# A* Algorithm with "Visit All Nodes" constraint
def a_star_visit_all(graph, start, goal):
    # Priority queue (min-heap) for storing and retrieving nodes with the lowest cost
    open_set = []
    all_nodes = set(graph.keys())  # Set of all nodes to visit
    initial_state = (0, start, frozenset([start]), [start], 0)  # (cost, current_node, visited_nodes, path, cost_without_heuristic)
    heapq.heappush(open_set, initial_state)

    # Store visited states (node + visited set) to avoid revisiting with a higher cost
    visited_states = {}

    while open_set:
        # Pop the node with the lowest cost
        current_cost, current_node, visited, path, cost_without_heuristic = heapq.heappop(open_set)

        # If all nodes have been visited and we reached the goal
        if visited == all_nodes and current_node == goal:
            return path, current_cost, cost_without_heuristic

        # Skip if this state has been visited with a lower cost
        state_key = (current_node, visited)
        if state_key in visited_states and visited_states[state_key] <= current_cost:
            continue
        visited_states[state_key] = current_cost

        # Explore neighbors of the current node
        for neighbor, weight in graph[current_node].items():
            new_cost = current_cost + weight
            new_cost_without_heuristic = cost_without_heuristic + weight  # Cost without heuristic
            new_visited = visited | frozenset([neighbor])
            heuristic = euclidean_heuristic(neighbor, goal)
            heapq.heappush(open_set, (new_cost + heuristic, neighbor, new_visited, path + [neighbor], new_cost_without_heuristic))

    # Return if no path is found
    return None, float('inf'), float('inf')

# Random Restart Hill Climbing with "Visit All Nodes" constraint
# Mention how changing max_restarts and max_steps can affect the output, like with this limits it fails to produce path for Canteen but produces for other starting points
def hill_climbing_tsp(graph, start, goal, nodes_to_visit, max_restarts=70, max_steps=20):
    best_path = None
    best_cost = float('inf')
    random.seed(42)
    
    for _ in range(max_restarts):
        # Start with a random path
        path = [start] + random.sample(nodes_to_visit, len(nodes_to_visit)) + [goal]
        total_cost = 0
        
        # Calculate the total cost for this path
        valid_path = True
        for i in range(len(path) - 1):
            current_node = path[i]
            next_node = path[i + 1]
            cost = graph.get(current_node, {}).get(next_node, None)  # Ensure no missing connections
            if cost is None:  # If no connection, the path is invalid
                valid_path = False
                break
            total_cost += cost

        if not valid_path:
            continue  # If the initial random path is invalid, skip to the next restart
        
        # Perform hill climbing: iteratively improve the path
        current_path = path
        current_cost = total_cost
        for _ in range(max_steps):
            neighbors = []
            # Try swapping each pair of nodes (excluding start and goal)
            for i in range(1, len(current_path) - 2):  # Exclude the start and goal nodes
                for j in range(i + 1, len(current_path) - 1):
                    # Swap nodes i and j
                    new_path = current_path[:]
                    new_path[i], new_path[j] = new_path[j], new_path[i]
                    
                    # Calculate the cost of the new path
                    new_cost = 0
                    valid_swap = True
                    for k in range(len(new_path) - 1):
                        node_a = new_path[k]
                        node_b = new_path[k + 1]
                        cost = graph.get(node_a, {}).get(node_b, None)
                        if cost is None:
                            valid_swap = False
                            break
                        new_cost += cost
                    
                    if valid_swap and new_cost < current_cost:
                        neighbors.append((new_path, new_cost))

            # If no valid neighbors found, break out of the loop
            if not neighbors:
                break
            
            # Choose the best neighbor (lowest cost)
            best_neighbor, best_neighbor_cost = min(neighbors, key=lambda x: x[1])
            current_path = best_neighbor
            current_cost = best_neighbor_cost
        
        # Update the best path if we found a better one
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = current_path
    
    return best_path, best_cost

# Function to evaluate the algorithm by measuring time, memory usage, and cost
def evaluate_algorithm(algorithm, graph, start, goal, nodes_to_visit=None):
    start_time = time.time()
    
    # For Hill Climbing, pass nodes_to_visit as an argument
    if algorithm == hill_climbing_tsp:
        path, total_cost = algorithm(graph, start, goal, nodes_to_visit)
    else:
        path, total_cost_with_heuristic, total_cost_without_heuristic = algorithm(graph, start, goal)
        total_cost = total_cost_without_heuristic
    
    end_time = time.time()
    time_taken = end_time - start_time
    memory_usage = sys.getsizeof(graph) + sys.getsizeof(path)  # Calculate memory usage
    return path, total_cost, time_taken, memory_usage

# Main function for user interaction and running algorithms
def main():
    print("Available locations:")
    for location in campus_map.keys():
        print(f"- {location}")

    # Take starting point as user input
    start_node = input("Enter the starting location: ").strip()
    goal_node = "Exit"  # Goal is always "Exit"

    if start_node not in campus_map:
        print("Invalid starting location! Please run the program again.")
        return

    # Nodes to visit (all except the start and goal node)
    nodes_to_visit = list(campus_map.keys())
    nodes_to_visit.remove(start_node)
    nodes_to_visit.remove(goal_node)

    # Run A* algorithm with "visit all nodes" constraint
    path, total_cost_with_heuristic, total_cost_without_heuristic = a_star_visit_all(campus_map, start_node, goal_node)

    if path:
        print(f"\nPath to visit all nodes and reach {goal_node}: {path}")
        print(f"Total cost with heuristic: {total_cost_with_heuristic}")
        print(f"Total cost without heuristic: {total_cost_without_heuristic}")
    else:
        print("\nNo path found!")

    # Run Random Restart Hill Climbing with "Visit All Nodes" constraint
    print("\nRunning Random Restart Hill Climbing (TSP)...")
    best_path, best_cost = hill_climbing_tsp(campus_map, start_node, goal_node, nodes_to_visit)
    if best_path:
        print(f"Best Path (Hill Climbing): {best_path}, Cost: {best_cost}")
    else:
        print("No valid path found for Hill Climbing.")

    # Evaluate A* algorithm
    print("\nEvaluating A* Algorithm...")
    _, a_star_cost, a_star_time, a_star_memory = evaluate_algorithm(a_star_visit_all, campus_map, start_node, goal_node)
    print(f"A* - Time: {a_star_time:.4f}s, Memory: {a_star_memory / 1024:.2f} KB, Cost: {a_star_cost}")

    # Evaluate Hill Climbing algorithm
    print("\nEvaluating Hill Climbing Algorithm...")
    _, hill_climbing_cost, hill_climbing_time, hill_climbing_memory = evaluate_algorithm(hill_climbing_tsp, campus_map, start_node, goal_node, nodes_to_visit)
    print(f"Hill Climbing - Time: {hill_climbing_time:.4f}s, Memory: {hill_climbing_memory / 1024:.2f} KB, Cost: {hill_climbing_cost}")


if __name__ == "__main__":
    main()


Available locations:
- Admission Office
- Hostel Office
- Hostel Visit
- Canteen
- Dep't Visit
- Library
- Exit


Enter the starting location:  Dep't Visit



Path to visit all nodes and reach Exit: ["Dep't Visit", 'Canteen', 'Library', 'Admission Office', 'Hostel Office', 'Hostel Visit', 'Exit']
Total cost with heuristic: 64.11787010740053
Total cost without heuristic: 21

Running Random Restart Hill Climbing (TSP)...
Best Path (Hill Climbing): ["Dep't Visit", 'Library', 'Admission Office', 'Hostel Office', 'Hostel Visit', 'Canteen', 'Exit'], Cost: 25

Evaluating A* Algorithm...
A* - Time: 0.0035s, Memory: 0.38 KB, Cost: 21

Evaluating Hill Climbing Algorithm...
Hill Climbing - Time: 0.0018s, Memory: 0.38 KB, Cost: 25
